### Загружаем YOLO

In [1]:
!pip install opencv-python-headless torch torchvision torchaudio
!git clone https://github.com/ultralytics/yolov5
!wget https://github.com/ultralytics/yolov5/releases/download/v6.2/yolov5s.pt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

### По сегментации делаем разметку

In [3]:
import cv2
import os
import numpy as np

image_folder = '/content/drive/MyDrive/Проекты/Мультитрекинг однородных объектов/YOLOv5-QCB+SORT/data/val/images'
mask_folder = '/content/drive/MyDrive/Проекты/Мультитрекинг однородных объектов/YOLOv5-QCB+SORT/data/val/masks'
output_folder = '/content/drive/MyDrive/Проекты/Отслеживание в реальном времени/Sort + other methods time/labels'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def extract_bounding_boxes(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boxes = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        boxes.append([x, y, x+w, y+h])
    return boxes

for image_name in os.listdir(image_folder):
    if image_name.endswith('.jpg'):
        image_path = os.path.join(image_folder, image_name)
        mask_path = os.path.join(mask_folder, image_name.replace('.jpg', '.png'))

        if not os.path.exists(mask_path):
            print(f"Маска для {image_name} не найдена. Пропуск...")
            continue

        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if image is None:
            print(f"Ошибка при загрузке изображения: {image_path}")
            continue
        if mask is None:
            print(f"Ошибка при загрузке маски: {mask_path}")
            continue

        boxes = extract_bounding_boxes(mask)
        h, w = image.shape[:2]
        txt_path = os.path.join(output_folder, image_name.replace('.jpg', '.txt'))
        with open(txt_path, 'w') as f:
            for box in boxes:
                x1, y1, x2, y2 = box
                x_center = (x1 + x2) / 2 / w
                y_center = (y1 + y2) / 2 / h
                width = (x2 - x1) / w
                height = (y2 - y1) / h
                f.write(f"0 {x_center} {y_center} {width} {height}\n")

        print(f"Разметка для {image_name} сохранена в {txt_path}")

Разметка для frame1_1100.jpg сохранена в /content/drive/MyDrive/Проекты/Отслеживание в реальном времени/Sort + other methods time/labels/frame1_1100.txt
Разметка для frame1_400.jpg сохранена в /content/drive/MyDrive/Проекты/Отслеживание в реальном времени/Sort + other methods time/labels/frame1_400.txt
Разметка для frame1_0.jpg сохранена в /content/drive/MyDrive/Проекты/Отслеживание в реальном времени/Sort + other methods time/labels/frame1_0.txt
Разметка для frame_1700.jpg сохранена в /content/drive/MyDrive/Проекты/Отслеживание в реальном времени/Sort + other methods time/labels/frame_1700.txt
Разметка для frame_1600.jpg сохранена в /content/drive/MyDrive/Проекты/Отслеживание в реальном времени/Sort + other methods time/labels/frame_1600.txt
Разметка для frame_1500.jpg сохранена в /content/drive/MyDrive/Проекты/Отслеживание в реальном времени/Sort + other methods time/labels/frame_1500.txt
Разметка для frame_1400.jpg сохранена в /content/drive/MyDrive/Проекты/Отслеживание в реальном в

### Обучаем YOLO

In [ ]:
data_yaml = """
train: /content/drive/MyDrive/Проекты/Мультитрекинг однородных объектов/YOLOv5-QCB+SORT/data/train/images
val: /content/drive/MyDrive/Проекты/Мультитрекинг однородных объектов/YOLOv5-QCB+SORT/data/val/images

nc: 1
names: ['bubble']
"""

data_path = '/content/data.yaml'

with open(data_path, 'w') as f:
    f.write(data_yaml)

In [ ]:
import os
import subprocess

data_yaml = "data.yaml"
model_weights = "yolov5s.pt"
epochs = 50
batch_size = 10
conf_thres = 0.1
max_det = 1000

command = f"python ./yolov5/train.py --data {data_yaml} --cfg yolov5s.yaml --weights {model_weights} --epochs {epochs} --batch-size {batch_size} --hyp hyp.scratch.yaml --img 640 --device 0 --conf-thres {conf_thres} --max-det {max_det}"

subprocess.run(command, shell=True)

In [ ]:
import time

start_time = time.time()

!yolo train data=data.yaml model=yolov5s.pt epochs=50 batch=10 conf=0.1 max_det=1000

end_time = time.time()
print(f"Время тренировки: {end_time - start_time:.2f} секунд")

### Детекция с помощью YOLO

In [ ]:
import time
import cv2
import os
from ultralytics import YOLO

model = YOLO('/content/runs/detect/train/weights/best.pt')
video_path = '/content/drive/MyDrive/Проекты/Мультитрекинг однородных объектов/YOLOv5-QCB+SORT/input_yolov5.mp4'
detections_folder = '/content/drive/MyDrive/Проекты/Отслеживание_в_реальном_времени/Sort+other_methods_time/Видео/детекции_пузыри_3/'
os.makedirs(detections_folder, exist_ok=True)

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Ошибка при открытии видео")
    exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
output_path = '/content/drive/MyDrive/Проекты/Мультитрекинг однородных объектов/YOLOv5-QCB+SORT/out_yolov5.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

frame_number = 0
total_time = 0
frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()
    results = model(frame, max_det=1000)
    end_time = time.time()
    inference_time = end_time - start_time
    total_time += inference_time
    frame_count += 1

    print(f"Кадр {frame_number}: время инференса = {inference_time:.4f} сек")

    annotations = results[0].boxes.data.cpu().numpy()

    if annotations.size > 0:
        detections_path = os.path.join(detections_folder, f"frame_{frame_number:05d}.txt")
        with open(detections_path, "w") as f:
            for box in annotations:
                x1, y1, x2, y2, conf, class_id = box
                width = x2 - x1
                height = y2 - y1
                f.write(f"{int(class_id)},{x1},{y1},{width},{height},{conf}\n")
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    out.write(frame)
    frame_number += 1

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"\nОбщее время: {total_time:.4f} сек")
if frame_count > 0:
    avg_time = total_time / frame_count
    print(f"\nСреднее время инференса: {avg_time:.4f} сек/кадр\n")
    print(f"FPS инференса: {1 / avg_time:.2f} кадров/сек\n")
else:
    print("Не удалось обработать ни одного кадра.")
print(f"Детекции для каждого кадра сохранены в {detections_folder}")


0: 480x640 202 bubbles, 43.9ms
Speed: 5.0ms preprocess, 43.9ms inference, 151.0ms postprocess per image at shape (1, 3, 480, 640)
Кадр 0: время инференса = 1.7489 сек

0: 480x640 198 bubbles, 12.1ms
Speed: 6.8ms preprocess, 12.1ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
Кадр 1: время инференса = 0.0268 сек

0: 480x640 188 bubbles, 12.1ms
Speed: 4.8ms preprocess, 12.1ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
Кадр 2: время инференса = 0.0255 сек

0: 480x640 197 bubbles, 12.1ms
Speed: 6.5ms preprocess, 12.1ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
Кадр 3: время инференса = 0.0284 сек

0: 480x640 181 bubbles, 12.0ms
Speed: 7.1ms preprocess, 12.0ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
Кадр 4: время инференса = 0.0266 сек

0: 480x640 204 bubbles, 14.0ms
Speed: 3.9ms preprocess, 14.0ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
Кадр 5: время инференса = 0.0268 сек

0

### Устанавливаем все необходимое, чтобы открыть расширение .mkv

In [ ]:
!apt-get update
!apt-get install -y ffmpeg

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,657 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,523 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,606 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/

In [ ]:
!ffmpeg -i "/content/42.ФМ ь 23-001.mkv"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
# !ffmpeg -i "/content/42.ФМ ь 23-001.mkv" -ss 00:00:00 -to 00:00:08 -c copy "/content/new_video1.mkv" #обрезка видео

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
!ffmpeg -i "/content/42.ФМ ь 23-001.mkv" -ss 00:00:00 -to 00:00:08 -c:v libx264 -preset fast -crf 23 -c:a aac -b:a 128k "/content/video_cut1.mp4"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

### Установка SORT

In [ ]:
!git clone https://github.com/abewley/sort.git

Cloning into 'sort'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 208 (delta 2), reused 1 (delta 1), pack-reused 203 (from 2)
Receiving objects: 100% (208/208), 1.21 MiB | 3.36 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [ ]:
!pip install filterpy==1.4.5
!pip install scikit-image==0.18.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=37e7e6dffbd71baff17d68ca29d6a1f0dde82730348a69930b024ee14548a7f9
  Stored in directory: /root/.cache/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 16.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 94.8 MB/s eta 0:00:00
  Created wheel for scikit-image: filename=scikit_image-0.18.1-cp311-cp311-linux_x86_64.whl size=37878414 sha256=6dacb834ec11e0eeb70d4b271b9faf04943eb03d66f0cbffcc5e16d9317df205
  Stored in directory: /root/.cache/pip/wheels/88/b9/72/a10a1db01af67d320f1fea8a4c82fa95733db1349315a5b81e
Successfu

### Установка lap

In [ ]:
!python -m pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
!pip install --upgrade lap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.8 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/gatagat/lap.git

Cloning into 'lap'...
remote: Enumerating objects: 958, done.
remote: Counting objects: 100% (410/410), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 958 (delta 276), reused 265 (delta 196), pack-reused 548 (from 1)
Receiving objects: 100% (958/958), 1.69 MiB | 5.66 MiB/s, done.
Resolving deltas: 100% (575/575), done.


In [ ]:
import lap

### Делим видео на кадры

In [ ]:
import cv2
import os

def split_video_into_frames(video_path, frames_folder):
    os.makedirs(frames_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_number = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(frames_folder, f"frame_{frame_number:05d}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_number += 1

    cap.release()
    print(f"Видео разделено на {frame_number} кадров и сохранено в папке {frames_folder}")

split_video_into_frames('/content/drive/MyDrive/Проекты/Мультитрекинг однородных объектов/YOLOv5-QCB+SORT/out_yolov5.mp4', '/content/drive/MyDrive/Проекты/Отслеживание_в_реальном_времени/Sort+other_methods_time/Видео/frame_detection')

Видео разделено на 100 кадров и сохранено в папке /content/drive/MyDrive/Проекты/Отслеживание_в_реальном_времени/Sort+other_methods_time/Видео/frame_detection


In [ ]:
!pip install filterpy

In [ ]:
!python /content/sort.py

Frame 0: 
  Avg IoU: 0.000
  Avg Optical Flow: 0.000
  Object Recall: 0.000
  New bubbles: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202}
  Disa

In [ ]:
!time python /content/drive/MyDrive/Проекты/Отслеживание_в_реальном_времени/Sort+other_methods_time/sort.py

Средний вектор перемещения: [-4.08098203  0.2914086 ]

real	0m20.316s
user	0m15.600s
sys	0m1.868s


### Наносим трекинг на изначальную видеозапись

In [ ]:
import os
import cv2
number_of_frames = 100
def visualize_tracking(video_path, tracked_data_folder, output_video_path):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))

    for frame_number in range(number_of_frames):
        ret, frame = cap.read()
        if not ret:
            break

        tracked_path = os.path.join(tracked_data_folder, f"tracked_{frame_number:05d}.txt")
        if os.path.exists(tracked_path):
            with open(tracked_path, "r") as f:
                for line in f:
                    x1, y1, x2, y2, obj_id = map(float, line.strip().split(","))
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                    cv2.putText(frame, f"ID: {int(obj_id)}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        out.write(frame)

    cap.release()
    out.release()

visualize_tracking('/content/drive/MyDrive/Проекты/Мультитрекинг однородных объектов/YOLOv5-QCB+SORT/input_yolov5.mp4','/content/drive/MyDrive/Проекты/Отслеживание_в_реальном_времени/Sort+other_methods_time/finish','/content/last_video_cut1.mp4')